# Understanding Real-Time Endpoints Architecture
Real-time endpoints represent SageMaker's solution for applications that require persistent, high-performance inference infrastructure. Unlike the serverless endpoints you've worked with in previous lessons, real-time endpoints maintain dedicated compute instances that remain active and ready to serve predictions at all times. This architectural approach eliminates the cold start delays that can occur with serverless endpoints when they scale from zero, ensuring that your model can respond to prediction requests with consistent, predictable latency.

The key architectural difference lies in the infrastructure persistence model. When you deploy a real-time endpoint, SageMaker provisions dedicated EC2 instances that host your model continuously. These instances run your model in memory, maintaining the loaded state and ready-to-serve configuration that enables immediate response to incoming requests. This persistent architecture makes real-time endpoints particularly well-suited for applications that require guaranteed response times, handle steady streams of prediction requests, or serve as critical components in time-sensitive decision-making processes.

Real-time endpoints also provide sophisticated auto-scaling capabilities that allow them to automatically adjust the number of running instances based on incoming traffic patterns. When request volume increases, SageMaker can automatically launch additional instances to maintain performance levels. When traffic decreases, the service can scale down while maintaining a minimum number of instances to ensure availability. This auto-scaling behavior provides the flexibility to handle traffic spikes while maintaining cost efficiency during lower-demand periods.

The use cases for real-time endpoints span across industries and applications where immediate responses are critical. Financial services use real-time endpoints for fraud detection systems that must evaluate transactions within milliseconds. E-commerce platforms deploy recommendation engines that provide instant product suggestions as users browse. Healthcare applications rely on real-time endpoints for diagnostic assistance tools that support clinical decision-making. Manufacturing systems use real-time endpoints for quality control processes that must evaluate products as they move through production lines. In each of these scenarios, the consistent low latency and high availability provided by real-time endpoints are essential for the application's success.

# Finding and Preparing the Trained Model


In [ ]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# List most recent completed training job
training_jobs = sagemaker_session.sagemaker_client.list_training_jobs(
    SortBy='CreationTime',                 # Sort jobs by creation time
    SortOrder='Descending',                # Newest jobs first
    StatusEquals='Completed',              # Only include completed jobs
    NameContains='sagemaker-scikit-learn'  # Filter to sklearn estimator jobs
)

# Extract the name of the latest training job of the list
TRAINING_JOB_NAME = training_jobs['TrainingJobSummaries'][0]['TrainingJobName']

# Configuring Real-Time Endpoint Resources


In [ ]:
# Unique name to assign to the deployed real-time endpoint
ENDPOINT_NAME = "california-housing-realtime"
# Type of instance to use for the endpoint
INSTANCE_TYPE = "ml.m5.large"
# Number of instances to launch for the endpoint
INSTANCE_COUNT = 1

# Executing the Deployment and Monitoring Status


In [ ]:
# Attach to existing training job
estimator = SKLearn.attach(TRAINING_JOB_NAME)

# Deploy as real-time endpoint
predictor = estimator.deploy(
    initial_instance_count=INSTANCE_COUNT,
    instance_type=INSTANCE_TYPE,
    endpoint_name=ENDPOINT_NAME,
    wait=False
)

# Retrieve detailed information about the specified SageMaker endpoint
endpoint_description = sagemaker_session.sagemaker_client.describe_endpoint(EndpointName=ENDPOINT_NAME)

# Extract the current status of the endpoint from the response
status = endpoint_description['EndpointStatus']

# Display the endpoint's status
print(f"Endpoint status: {status}")

# Considerations for Real-Time Endpoint Usage

Before deploying real-time endpoints in production, you must understand the key trade-offs that will impact both performance and costs. These considerations help determine when real-time endpoints are the optimal choice for your specific use case.

- Cost implications: Real-time endpoints incur continuous costs for all provisioned instances regardless of usage, unlike serverless endpoints that only charge for actual requests. This makes them most economical for steady, predictable traffic patterns.

- Instance selection and sizing: Balance your model's resource requirements against costs. Instances like ml.m5.large provide a good balance of performance and cost for many workloads, while smaller instances like ml.t3.medium may be more economical for lightweight models. Larger instances like ml.m5.xlarge or ml.c5.2xlarge provide more power at proportionally higher costs. GPU-enabled instances (ml.p3 or ml.g4dn families) may be necessary for deep learning models but significantly increase costs.

- Scaling and availability: Production deployments typically need multiple instances for redundancy and traffic handling. Auto-scaling takes time as new instances must launch and initialize, requiring proactive rather than reactive scaling policies.

- Monitoring and maintenance: Real-time endpoints require continuous monitoring of instance health, performance metrics, and scaling behavior. Model updates need careful coordination across running instances to minimize downtime.

These considerations help you choose between serverless and real-time endpoints based on your traffic patterns, latency requirements, and budget constraints rather than technical preferences alone.